In [1]:
import pandas as pd
import numpy as np
import nltk as nlp
from nltk.corpus import stopwords
from string import punctuation
from sklearn.feature_extraction.text import CountVectorizer

### Preprocessing

In [2]:
def norm(text):
    text = text.translate(str.maketrans('', '', punctuation))
    text = text.lower()
    text = nlp.word_tokenize(text)
    stop_words = set(stopwords.words('russian'))
    text = [word for word in text if word not in stop_words]
    lemmatizer = nlp.WordNetLemmatizer()
    roots = [lemmatizer.lemmatize(each) for each in text]

    return ' '.join(roots)

In [3]:
data = pd.read_csv('texts.csv').values.tolist()
data = [x[0] for x in data]

In [4]:
processed = map(norm, data)

### BOW

In [5]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(processed)
df = pd.DataFrame(X.todense().tolist(), columns=vectorizer.get_feature_names())
df.to_csv('processed_nlp.csv', index=False)

### Table cleaning

In [6]:
data = pd.read_csv('processed_nlp.csv')

In [7]:
data.head()

,00,000,0000,000006740,000006740лчек,0000410314854,0001,0016,00173,0027,...,ястребовым,ясушин,ясушинмихаил,ященко,ящик,ящика,ящике,ящики,ёмкости,ёмкость
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
to_drop = [] # columns with numbers
for i in range(2007):
    to_drop.append(data.columns[i])
data.drop(to_drop, axis=1, inplace=True)

In [9]:
data.head()

,аа,аамо,абз,абзацах,абонемент,абонемента,абонент,абонента,абонентов,абонентом,...,ястребовым,ясушин,ясушинмихаил,ященко,ящик,ящика,ящике,ящики,ёмкости,ёмкость
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
# word length <= 2
to_drop = []
for i, val in enumerate(data.columns):
    if len(val) <= 2:
        to_drop.append(data.columns[i])
data.drop(to_drop, axis=1, inplace=True)

In [11]:
data.head()

,аамо,абз,абзацах,абонемент,абонемента,абонент,абонента,абонентов,абонентом,абонентская,...,ястребовым,ясушин,ясушинмихаил,ященко,ящик,ящика,ящике,ящики,ёмкости,ёмкость
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### LSI

In [12]:
U, S, Vh = np.linalg.svd(data.values, full_matrices=False)

In [13]:
num_topics = 11

In [14]:
# truncating to the number of topics
U = U[:,:num_topics] # docs x topics
S = S[:num_topics] # topics x topics
Vh = Vh[:num_topics,:] # topics x terms

In [15]:
# printing 10 most probable words for each topic
for i, row in enumerate(Vh):
    ind = np.argsort(row)[-10:]
    print('\nTopic №%d:' % (i+1))
    topic_words = ''
    for word_id in ind:
        topic_words += data.columns[word_id] + ' '
    print(topic_words[:-1])


Topic №1:
дизельных очистить форсункиa обед микрорайон рвота бизнесланча дыхания водолазов предварительная

Topic №2:
мои ответ рублей деньги добрый прошу года день ооо это

Topic №3:
платежа переулок сухаревский кпп посылки огрн инн пим москва ооо

Topic №4:
продавец возврате качества магазина добрый магазине день магазин вернуть товар

Topic №5:
руб смс срок ноября декабря доставки кровать заказа года заказ

Topic №6:
ноября заказа санпин декабря продукции соответствии доставки товар продукция заказ

Topic №7:
пао товар просто ооо банк форекс рублей банка сайте это

Topic №8:
рублей суда года деньги автомобиля ооо продавец вернуть автомобиль товар

Topic №9:
это сайта автомобиль мтс просто сайт домен день данные года

Topic №10:
банка продукция мои теле2 данных оператор заявление нужно мтс данные

Topic №11:
заявление дней ооо услуги магазин магазине нужно данные мтс это
